In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse.linalg import eigs, eigsh
from scipy.linalg import eig, eigh
from functools import reduce
import pickle
import qutip as qt
import jax
# from jax import config; config.update('jax_enable_x64', True) # config.update('jax_platform_name', 'cpu')
import jax.numpy as jnp

from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion.ops import InteractionOperator, FermionOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator, eigenspectrum

from pyscf import gto, dft, scf, cc, df, ao2mo, fci
# from pyscf import fci

import pennylane as qml
import pennylane.numpy as qmlnp
from pennylane import qchem

from ham import(
    pauli, get_qml_ham
)

from utils import(
    PauSumHam, taper, bit_count
)

from tapering import(
    get_ao_rep,
    get_generators
)

bohr = 0.529177249


#### test

In [2]:
# try: jnp.arange(2)
# except Exception as e:
#     print(e)
#     jax.config.update('jax_platform_name', 'cpu')

## Structure

In [3]:
NH3_atom = [['N' ,  ( 0.0000000,  0.0000000,  0.1493220)],
			['H' ,  ( 0.0000000,  0.9474830, -0.3484190)],
			['H' ,  ( 0.8205440, -0.4737420, -0.3484190)],
			['H' ,  (-0.8205440, -0.4737420, -0.3484190)]]

np.array([coord for atom, coord in NH3_atom])

array([[ 0.      ,  0.      ,  0.149322],
       [ 0.      ,  0.947483, -0.348419],
       [ 0.820544, -0.473742, -0.348419],
       [-0.820544, -0.473742, -0.348419]])

In [4]:
symbols = ["N", "H", "H", "H"]
coordinates = qmlnp.array([coord for atom, coord in NH3_atom], requires_grad=False)
n_electrons = 10

## Hamiltonian

In [5]:
qham, n_qubits = qchem.molecular_hamiltonian(symbols, coordinates, basis='sto-3g', mapping='jordan_wigner')

### generators

In [6]:
generators = qml.symmetry_generators(qham)
paulixops = qml.paulix_ops(generators, n_qubits)
paulix_sector = qml.qchem.optimal_sector(qham, generators, n_electrons)

/home/physics/.local/lib/python3.8/site-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/home/physics/.local/lib/python3.8/site-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


In [7]:
generators

[<Hamiltonian: terms=1, wires=[0, 2, 4, 6, 8, 10, 12, 14]>,
 <Hamiltonian: terms=1, wires=[1, 3, 5, 7, 9, 11, 13, 15]>]

### qham tapering

In [8]:
# qham_tapered = qml.taper(qham, generators, paulixops, paulix_sector)
# over 4 min, memory not enough

In [9]:
# E_qml_tapered = eigsh(qml.utils.sparse_hamiltonian(qham_tapered), k=1, which='SA')[0]

### psh ham tapering

In [10]:
# psham_tapered = taper(qham, generators, paulixops, paulix_sector)

In [11]:
# E_psh_tapered = eigsh(psham_tapered.to_sparse(), k=1, which='SA')[0][0] # take over 30 min
# E_psh_tapered = -14.82514221821951

In [12]:
# E_psh_tapered

In [13]:
# n_qubits, psham_tapered.n_qubits # (14, 10)

### point-group symmetry tapering (with psh)

In [14]:
# n_basis, params = qchem.mol_basis_data('sto-3g', symbols)
# alpha = qmlnp.array([params[i][1] for i in range(len(symbols))])
# alpha, n_basis, params

In [15]:
mol = qchem.Molecule(symbols, coordinates) # , alpha=alpha


In [16]:
qchem.scf(mol)() # alpha
mo_coeff = mol.mo_coefficients
n_ao = mo_coeff.shape[0]

In [17]:
# with open('BeH2_mo_coeff.pickle', 'rb') as f:
#     mo_coeff = pickle.load(f)

In [18]:
np.round(mo_coeff, 2)

tensor([[-0.99, -0.24,  0.  , -0.  , -0.1 , -0.14,  0.  , -0.  ],
        [-0.07,  0.91, -0.  , -0.  ,  0.28,  3.64, -0.  ,  0.  ],
        [ 0.  ,  0.  , -0.  , -0.71,  0.  ,  0.  ,  1.6 , -0.  ],
        [ 0.  , -0.  , -0.71,  0.  , -0.  , -0.  , -0.  , -1.6 ],
        [ 0.02, -0.4 ,  0.  ,  0.  ,  0.95, -1.02,  0.  , -0.  ],
        [ 0.02,  0.01, -0.42,  0.  ,  0.06, -1.52,  0.  ,  2.16],
        [ 0.02,  0.01,  0.21, -0.36,  0.06, -1.52, -1.87, -1.08],
        [ 0.02,  0.01,  0.21,  0.36,  0.06, -1.52,  1.87, -1.08]], requires_grad=True)

In [19]:
with open('NH3_ao_info.pickle', 'rb') as f:
    ao_info = pickle.load(f)

with open('NH3_rotations.pickle', 'rb') as f:
    rotations = pickle.load(f)

# BeH2_rotations

In [20]:
# ao_info
# rotations

In [21]:
ao_rep_list = get_ao_rep(n_ao, rotations, *ao_info, mol.coordinates)

In [33]:
# ao_rep_list

In [23]:
def get_mo_sym(ao_rep_list, mo_coeff):
    N = len(ao_rep_list)
    M = mo_coeff.shape[1]

    sym_list = [] # np.empty((N, M), dtype=np.int32)
    for i in range(N):
        sym_array = np.empty(M, dtype=np.int32)
        sym_tag = True
        for j in range(M):
            org_vec = mo_coeff[:, j]
            rep_vec = ao_rep_list[i] @ org_vec
            # print(org_vec, rep_vec)
            # if   np.allclose(rep_vec,  org_vec, rtol=1e-3, atol=1e-6): sym_array[j] =  1
            # elif np.allclose(rep_vec, -org_vec, rtol=1e-3, atol=1e-6): sym_array[j] = -1
            if   (np.abs(rep_vec - org_vec) <= 1e-2).all(): sym_array[j] =  1
            elif (np.abs(rep_vec + org_vec) <= 1e-2).all(): sym_array[j] = -1
            else:
                # print(sym_array)
                sym_tag = False
                break
        if sym_tag:
            # print(sym_array)
            sym_list.append(sym_array)
    
    return sym_list

mo_sym_list = get_mo_sym(ao_rep_list, mo_coeff)
mo_sym_list

[array([ 1,  1,  1, -1,  1,  1, -1,  1], dtype=int32)]

In [24]:
np.kron(mo_sym_list[0], np.array([1, 1]))

array([ 1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1])

In [25]:
def get_kernel(ao_rep_list, mo_coeff):
    from pennylane.qchem.tapering import _reduced_row_echelon

    n_mo = mo_coeff.shape[1]
    mo_sym_list = get_mo_sym(ao_rep_list, mo_coeff)

    spin_arr = np.array([1, -1], dtype=np.int32)
    mo_arr = np.ones(n_mo, dtype=np.int32)
    spin_swap_list = [np.kron(mo_arr, spin_arr), np.kron(mo_arr, -spin_arr)]
    so_sym_list = spin_swap_list + [np.kron(mo_sym, np.ones(2, dtype=np.int32)) for mo_sym in mo_sym_list]

    so_sym_array = np.array(so_sym_list)
    kernel = np.where(so_sym_array == 1, 0, 1)
    kernel = _reduced_row_echelon(kernel)
    kernel = np.delete(kernel, np.where(np.sum(kernel, 1) == 0),axis=0)
    return kernel


In [26]:
kernel = get_kernel(ao_rep_list, mo_coeff)
kernel

array([[1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0]])

In [27]:
generators_pgs = get_generators(kernel)
paulixops_pgs = qml.paulix_ops(generators_pgs, n_qubits)
paulix_sector_pgs = qml.qchem.optimal_sector(qham, generators_pgs, n_electrons)

In [28]:
# psham_pgs_tapered = taper(qham, generators_pgs, paulixops_pgs, paulix_sector_pgs)

In [29]:
# E_psh_pgs_tapered = eigsh(psham_pgs_tapered.to_sparse(), k=1, which='SA')[0][0] # take ? min

In [30]:
# E_psh_pgs_tapered, E_psh_tapered

In [31]:
len(generators), len(generators_pgs)

(2, 3)

In [32]:
# n_qubits, psham_pgs_tapered.n_qubits

pgs-tapering method can get 3 generators.